# Analysis Ribo-TIS reads

Detection of sample-specific Translation Initiation Sites (TIS) from aligned Ribo-TIS reads.

<b>TIS calling:</b> to detect sample-specific Translation Initiation Sites (TIS) from the aligned Ribo-TIS reads, we developed a probabilistic approach to estimate a confidence score used to identify the genomic positions of putative start codons. 

## Estimation Probability

We assumed that all annotated start codons aligning with Ribo-TIS reads were true start codons, from this we propose to estimate the probability of each position $(pos)$  into each read length $l=\left(26,...,34\right)$, to act as the first nt of the ribosomal P-site therefore being the first nt of a start codon $(sc)$, as follows:

$<1>$
Let 

$ r=\ \left\{\mathrm{reads\ being\ at\ the\ first\ nt\ of\ a\ start\ codon}\middle|\mathrm{\ }len{=}l,\ then\ pos{=}p\right\}$

$R=\ \left\{\mathrm{total\ reads\ being\ at\ the\ first\ nt\ of\ a\ start\ codon}\middle| l e n{=}l\right\};$

$$P\left(sc\ \middle|\ len=\ l,\ pos=p\ \right)\ =\ \frac{|r|}{|R|}$$

where $P(sc| len= l, pos=p)$ is the probability of a sc at the read position pos in the read of length $l=(26,...,34)$ is the probability of a sc at the read position pos in the read of length $l=(26,...,34)$.


Then, we computed two heuristics to evaluate the certainty of the ribosomal P-site location into each read length l, and the relevance of the read-alignment regarding its multimapping. 
The first heuristic $H_1\left(l\right)$ assigned a normalized weight to each read length $(26-34 nt)$, computed through the standard deviation of the read positions acting as start codons, as follows:


$<2>$
Let 

$\sigma\ =\ \left\{\sigma_l\middle|\ stdev\ of\ read\ positions\ acting\ as\ start\ codons\ for\ l=\left(26,\ldots,34\right)\right\}$

$$H_1\left(l\right)=1-\left(\frac{\sigma_l-\min{\left(\sigma\right)}}{\max{\left(\sigma\right)}-\min{\left(\sigma\right)}}\ast0.99\right)$$

The second heuristic $H_2\left(R_r\right)$ assigned a weight to each Ribo-Tis read according to its rank $\left(R_r\right)$  assigned a weight to each Ribo-Tis read according to its rank $\left(R_r\right)$ in which STAR has reported such alignments, as follows:

$<3>$

$$H_2\left(R_r\right)=1-\left(\frac{R_r-1}{max_R-1}\ast0.99\right)$$

where $max_R$ is the max number of hits reported by STAR (default = 10).

Input files: 
```python
"""
nameTask : string
    qsub name task 
saveOutputQsub : string - path
    Path to save Qsub output
starCodonsPositions : path to list
    Bed list of annotated start codons
allStarCodonsPositions : path to dic
    Dic of annotated start codons
bamFile : path to Bam File
    Deduplicated Bam File
toSaveOutPuts : path
    Path to save output files
minReads : int
    Minimum reads: a known start codon must have at least 3 reads to be considered
logPath : string
    Path to save logs
"""
```
Output Files:
```python
"""
Known_scs_intercepted : dic
    Dic with information of the annotated start codons found in the sample (intercepted by Ribo-Tis reads)
Known_scs_intercepted : list
    List with information of the annotated start codons found in the sample (intercepted by Ribo-Tis reads)
frequenciesByLenFragment : dic
    Path to the dic of frequencies of each position of a read of len (26-34ntd), 
    where a known start codon is found
Known_SC_Intercepted_Forward : bed-like file
    Bed file with the start codons intercepted in the Forward strand
Known_SC_Intercepted_Backward : bed-like file
    Bed file with the start codons intercepted in the Backward strand

probasByLenFragmentFromFrequencies : dic
    Dic with the computed probability for each len (26-34) 
infoStdev : dic
    Dic with information for each len read (26-34ntd) : total reads, mean, median, std
weightsByLenght : dic
    Dic with the computed heuristique H_1(l) (see above) for each len (26-34)
"""
```

In [8]:
%%bash

module add torque 
nameTask='GetProbasFromAnnotatedSC'
saveOutputQsub='.../qsub_outputs'
starCodonsPositions='../../../Ribo_db/Data_Input_Scripts/Start_codon_Positions.bed'
allStarCodonsPositions='../../../Ribo_db/Data_Input_Scripts/all_start_codon_Positions.dic'
bamFile='.../Data_Preparation/Ribo_Seq_Data/Alignment_Reads_Genome/Ribo/TIS/...DD.bam' 
toSaveOutPuts='.../TIS_calling_Info/'
minReads=3 
logPath='.../logs/'

echo "../../Scripts/2_TIS_Calling/getProbasByLenFragmentTIS.py -i $bamFile -a $allStarCodonsPositions -o $toSaveOutPuts -s $starCodonsPositions -m $minReads -l $logPath" | qsub -V -l nodes=1:ppn=10,mem=50gb,walltime=48:00:00 -j oe -N $nameTask -d $saveOutputQsub


## Sam File from Bam File Ribo-Tis alignment

Generate Sam file fromt the Ribo-Tis Bam file : file to be use in the detection TIS step.

Input Files
```bash
"""
bamFile : path to Bam File
    Deduplicated Bam File
samFile : path to save Sam File
    Path to save sam file
nameTask : string
    qsub name task
saveOutputQsub : path
    Path to save qsub files
logPath : path
    Path to save log file
"""
```

In [18]:
%%bash
module add samtools
module add torque

bamFile='.../Data_Preparation/Ribo_Seq_Data/Alignment_Reads_Genome/Ribo/TIS/...DD.bam'
samFile='.../Data_Preparation/Ribo_Seq_Data/Alignment_Reads_Genome/Ribo/TIS/...DD.sam'
nameTask='BamToSam'
saveOutputQsub='.../qsub_outputs'
logPath='.../logs/'

echo "samtools view -h -o $samFile $bamFile" | qsub -V -l nodes=1:ppn=10,mem=50gb,walltime=48:00:00 -j oe -N $nameTask -d $saveOutputQsub


## Single Nucleotide Variants detection : FreeBayes

Detection of the SVP using FreeBayes tool

Input Files
```bash
"""
listFile : path to list
    List of bam files of each sample (Ribosome profiling and RNA seq)
genomePathFai : path
    Path to genome index
genomePath : path
    Path to genome fasta
outputFile : path
    Path to save output from FreeBayes
nameTask : string
    qsub name task
saveOutputQsub : path
    Path to save qsub output
logPath : path
    Path to save log output
"""
```

In [20]:
%%bash

echo 'Running FreeBayes Calling'
listFile='.../listBamFiles.txt'
genomePathFai='../../../Ribo_db/Data_Input_Scripts/GRCh38_Gencode26/GRCh38.primary_assembly.genome.fa.fai'
genomePath='../../../Ribo_db/Data_Input_Scripts/GRCh38_Gencode26/GRCh38.primary_assembly.genome.fa'
outputFile='.../FreeBayes/'
nameTask='FreeBayes_SNPsCalling'
saveOutputQsub='.../qsub_outputs'
logPath='.../logs/'

sh ../../../Ribo_db/Scripts/2_TIS_Calling/freeBayes.sh $listFile $genomePathFai $genomePath $outputFile $nameTask $saveOutputQsub $logPath


Running FreeBayes Calling


In [ ]:
%%bash

input='.../FreeBayes/output.var.5X.vcf'
python ../../Scripts/2_TIS_Calling/freebayesVCF_to_Agnostic.py $input 


# TIS Detection

## First step

Detection of all the start codons candidates according to Ribo-TIS reads.

This step takes the sam file alignments and search all the posible start codons (ATG+Near cogante codons) into them.

For each start codon, the score of start codon is calculated, as :

$$P\left(c\middle|\ Ribo-Tis\ reads\ mapped\ to\ x\right)=\frac{\sum_{r\ read}^{Ribo-Tis}{P\left(sc\ \middle|\ len\ =\ l,\ pos\ =\ p\ \right)}\cdot H_1\left(l\right)\cdot H_2\left(R_r\right)}{\sum_{r\ read}^{Ribo-Tis}{H_1\left(l\right)\cdot H_2\left(R_r\right)}}$$

Where:

where $x$ is the genomic position of the first nucleotide of a candidate start codon and $c$ is the event that indicates that the position $x$ is a start codon $sc$.


Input Files

```python
"""
logPath : path
    Parth to save logs output

getResults : boolean
    True --> to get all start codons candidates in the sample
minReads : Int
    Minimum reads: a start codon must have at least 3 reads to be considered

folderToSave : path
    Path to vfolder where the results will be saved
probasByLenFragmentFromFrequencies : dic
    Dic with the computed probability for each len (26-34) --  computed steps before
    
weightsByLenght : dic
    Dic with the computed heuristique H_1(l) (see above) for each len (26-34)

fastaFile : path
    Path to the genome fasta
filepath_index : path
    Path to the genome Index

samFile : path
    Path to the sam file for the Ribo-TIS alignment corresponding to the sample
freeByesSNPs : path
    Path to the FreeBayes output
quality : int
    Quality of the SVN (default:20)
"""
```

Output Files:

```python
"""
SortedStartCodonsAfterApplyFilterByMinReads : list 
    List from all the start codons identified only those that have a min reads bigger or equal to a min reads chosen from the user are keeped

AllStartCodonsWithoutFilter_SamFile : dic
    Dic that saves all the start codons identified
```

In [23]:
%%bash

module add torque

echo 'Start codons detection'

logPath='.../logs/'

getResults=True
minReads=3 

folderToSave='.../StartCodonsDetection/'
probasByLenFragment='.../TIS_calling_Info/probasByLenFragmentFromFrequencies.dic'
weightsByLenght='.../TIS_calling_Info/weightsByLenght.dic'

fastaFile='../../Data_Input_Scripts/GRCh38_Gencode26/GRCh38.primary_assembly.genome.fa'
filepath_index='../../Data_Input_Scripts/GRCh38_Gencode26/GRCh38.primary_assembly.genome.fa.fai'

samFile='.../Alignment_Reads_Genome/Ribo/TIS/...DD.sam'
freeByesSNPs='.../FreeBayes/output.var.5X.pga'
quality=20

python ../../../Ribo_db/Scripts/2_TIS_Calling/mainClassStartCodonsDetection.py -r $getResults -o $folderToSave -p $probasByLenFragment -w $weightsByLenght -m $minReads -g $fastaFile -i $filepath_index -s $samFile -l $logPath -f $freeByesSNPs -q $quality


Start codons detection


## Second step

From all the start codons candidates identified in the first step, we select those that are above the threshold.

A threshold on $P\left(pos\middle| c\right)$ is establish to retain only the start codons candidates with high confidence, we ranked the computed confidence results to plot a receiver operating characteristic curve (ROC curve). This curve is plotted using the known start codons as positives and any other start codon candidates as negatives. For each point on the curve, we computed the Euclidean distance to a perfect classifier $(0,1)$ and then reported the threshold corresponding to the shortest distance to that point. Thus, any start codon candidate whose computed confidence was above the threshold was considered as a positive start codon position and was retained for further analysis.

Input Files
```python
"""
logPath : path
    Path to save log file
getResults : boolean
    False --> gets the final start codons after the threshold has been applied

folderToSave :  path
    Path to folder to save outputs
Known_scs_intercepted : list
    List with information of the annotated start codons found in the sample (intercepted by Ribo-Tis reads)
"""
```
Output Files
```python
"""
ROC_courbe : png image
    Image of ROC. RIC is used to choose how many start codons have good probability

StartCodonsAfterThreshold : list 
    List of start codons after the score-threshold has been applied from the ROC courve
    
StartCodonsAfterThreshold : dic 
    Dic for the start codons after the threshold has been applied from the ROC courve

BedFileForward : bed-like file
BedFileBackward : bed-like file 
    Bed file of backward and forward strand start codons. These bed files will be use to intersect start codons with assembled transcripts

Canonical_SC : dic
    Dic resume of the canonical start codons with their score
"""
```

In [24]:
%%bash

module add torque

echo 'Start codons final candidates'

logPath='.../logs/'
getResults=False

folderToSave='.../StartCodonsDetection/'
retainedStartsKnown='.../TIS_calling_Info/Known_scs_intercepted.list'

python ../../../Ribo_db/Scripts/2_TIS_Calling/mainClassStartCodonsDetection.py -r $getResults -o $folderToSave -k $retainedStartsKnown -l $logPath


Start codons final candidates
